In [10]:
import sqlite3
from allocation.utilities import utility 
import pandas as pd
from datetime import datetime as dt
import collections
import numpy
import json
from cvxopt import matrix, solvers


start_date = "2007-01-02"
end_date = "2019-01-29"

In [11]:
conn = sqlite3.connect('db.sqlite3')
c = conn.cursor()

In [12]:
start_date_dt = dt.strptime(start_date, '%Y-%m-%d')
end_date_dt = dt.strptime(end_date, '%Y-%m-%d')

results = c.execute("SELECT codec FROM allocation_asset")
symbols = sorted([row[0] for row in results])
asset_series = dict()

q = """
     select date, price
     from allocation_series
     where asset_id=:symbol and 
     date between date(:start_date) and date(:end_date);
     """

for symbol in symbols:
    params = {'symbol': symbol, 'start_date': start_date, 'end_date': end_date}
    asset_series[symbol] = pd.read_sql(q, conn, params=params)
    if asset_series[symbol].iloc[0]["date"] != start_date or asset_series[symbol].iloc[len(asset_series[symbol])-1]["date"] != "2019-01-29":
        del asset_series[symbol]
        print symbol, "deleted"

asset_returns_an = collections.OrderedDict(sorted(utility.return_assets(asset_series, 
                                                                        start_date_dt, end_date_dt)[0].items()))
risk, cov = utility.risk_corr(asset_series, start_date_dt, end_date_dt)
print len(cov), len(a)
#print collections.OrderedDict(sorted(risk.items())), cov



ISCI137 deleted
ISCI325 deleted
ISCI326 deleted
ISCI328 deleted
ISCI40 deleted


ISCI693 deleted
ISCI830 deleted


In [34]:

cov_na = cov.to_numpy()
sigma = matrix(cov_na)
symbols, return_an = zip(*asset_returns_an.items())
#avg_r = map(lambda s: return_an, symbols)
avg_r = numpy.array(return_an)
r_min = 0.07
print len(avg_r), symbols
print len(sigma)

71 (u'IBSC520', u'ISCI10', u'ISCI11', u'ISCI110', u'ISCI111', u'ISCI112', u'ISCI113', u'ISCI114', u'ISCI115', u'ISCI129', u'ISCI132', u'ISCI134', u'ISCI14', u'ISCI140', u'ISCI143', u'ISCI144', u'ISCI145', u'ISCI150', u'ISCI152', u'ISCI156', u'ISCI158', u'ISCI160', u'ISCI163', u'ISCI164', u'ISCI174', u'ISCI175', u'ISCI176', u'ISCI193', u'ISCI194', u'ISCI196', u'ISCI197', u'ISCI199', u'ISCI2', u'ISCI204', u'ISCI205', u'ISCI25', u'ISCI251', u'ISCI26', u'ISCI27', u'ISCI28', u'ISCI29', u'ISCI30', u'ISCI31', u'ISCI32', u'ISCI329', u'ISCI33', u'ISCI34', u'ISCI4', u'ISCI41', u'ISCI45', u'ISCI46', u'ISCI48', u'ISCI49', u'ISCI5', u'ISCI6', u'ISCI62', u'ISCI679', u'ISCI7', u'ISCI71', u'ISCI74', u'ISCI76', u'ISCI8', u'ISCI80', u'ISCI81', u'ISCI85', u'ISCI86', u'ISCI87', u'ISCI88', u'ISCI9', u'ISCI92', u'ISCI94')
5041


In [87]:
n = len(avg_r)
returns = numpy.asmatrix(avg_r)

# Convert to cvxopt matrices
R = avg_r

P = sigma
q = matrix(numpy.zeros(n))
G = matrix(-numpy.eye(n))
h = matrix(numpy.zeros(n))
A = matrix(numpy.array([R, numpy.ones(n)]))
b = matrix(numpy.array([0.1, 1]))
print P

[ 2.49e-03  2.02e-03  1.59e-03  1.09e-03  1.27e-03  1.14e-03  1.30e-03 ... ]
[ 2.02e-03  2.18e-03  1.67e-03  1.29e-03  1.43e-03  1.36e-03  1.51e-03 ... ]
[ 1.59e-03  1.67e-03  1.48e-03  1.14e-03  1.25e-03  1.19e-03  1.36e-03 ... ]
[ 1.09e-03  1.29e-03  1.14e-03  1.41e-03  1.48e-03  1.49e-03  1.65e-03 ... ]
[ 1.27e-03  1.43e-03  1.25e-03  1.48e-03  1.62e-03  1.53e-03  1.72e-03 ... ]
[ 1.14e-03  1.36e-03  1.19e-03  1.49e-03  1.53e-03  1.64e-03  1.77e-03 ... ]
[ 1.30e-03  1.51e-03  1.36e-03  1.65e-03  1.72e-03  1.77e-03  2.16e-03 ... ]
[ 1.37e-03  1.54e-03  1.39e-03  1.65e-03  1.77e-03  1.73e-03  2.08e-03 ... ]
[ 1.25e-03  1.42e-03  1.25e-03  1.61e-03  1.66e-03  1.76e-03  2.08e-03 ... ]
[ 2.43e-06  2.65e-06 -2.96e-07  1.19e-06  1.35e-06  3.67e-07  1.64e-06 ... ]
[ 4.42e-04  7.67e-04  5.52e-04  4.16e-04  4.05e-04  4.53e-04  5.14e-04 ... ]
[ 2.98e-05  4.68e-04  3.88e-04  3.87e-04  3.76e-04  4.29e-04  4.59e-04 ... ]
[ 1.24e-03  1.38e-03  1.29e-03  1.08e-03  1.16e-03  1.14e-03  1.24e-03 ... ]

In [102]:
#P = matrix(numpy.array([[2.49e-03, 2.02e-03], [2.02e-03, 2.18e-03]]))
#q = matrix(numpy.zeros(2))
#G = matrix([[-1, 0], [0, -1]])
#h = matrix(numpy.ones(2))
#A = matrix([[1, 1], [0.05, 0.06]])
#b = matrix([1, 0.01])
P = matrix(numpy.diag([1,0]), tc='d')
q = matrix(numpy.array([3,4]), tc='d')
G = matrix(numpy.array([[-1,0],[0,-1],[-1,-3],[2,5],[3,4]]), tc='d')
h = matrix(numpy.array([0,0,-15,100,80]), tc='d')
print G
sol = solvers.qp(P, q, G, h)

[-1.00e+00  0.00e+00]
[ 0.00e+00 -1.00e+00]
[-1.00e+00 -3.00e+00]
[ 2.00e+00  5.00e+00]
[ 3.00e+00  4.00e+00]



ValueError: Rank(A) < p or Rank([P; A; G]) < n